# Youtube $\rightarrow$ iTunes

First, let's import some standard packages we might need

In [18]:
# misc
import os
import shutil
import math
import datetime
# plots
import matplotlib.pyplot as plt
%matplotlib inline


## Test Download Methods

### Method 1: Using PyTube

In [15]:
pip install pytube

Note: you may need to restart the kernel to use updated packages.


In [16]:
from pytube import YouTube

In [17]:
!conda list | grep pytube

pytube                    11.0.1                   pypi_0    pypi


Download **video** with `pytube`

In [5]:
video_url = 'https://www.youtube.com/watch?v=YWhSQpUNGgY'
# video.streams.filter(file_extension = "mp4").all()
youtube = YouTube(video_url)
video = youtube.streams.first()
video.download(filename="Sticks_and_Stones.mp4")
# video.filesize

'/Users/colbyr/Documents/GitHub/downloader/Sticks_and_Stones.mp4'

Download *only* **audio** with `pytube`

In [13]:
youtube_video_url = 'https://youtu.be/bIjCNQT5BNU'
track_title = '8hr Binarual Sleep.mp3'
 
yt_obj = YouTube(youtube_video_url)

/Users/colbyr/miniconda3/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
audio = yt_obj.streams.filter(only_audio=True).first()
audio.download('./downloads', filename=track_title)

'/Users/colbyr/Documents/GitHub/downloader/./downloads/8hr Binarual Sleep.mp3'

### Alternate Method (NOT USING)
This method employs the `requests` module, but will not be used because it does not seem to work consistently

In [ ]:
# import requests
# file_url = "https://www.youtube.com/watch?v=YWhSQpUNGgY"
# file_name = "Sticks and Stone.mp4"

# r = requests.get(file_url, stream = True)

# # download started 
# with open(file_name, 'wb') as f: 
#     for chunk in r.iter_content(chunk_size = 1024*1024): 
#         if chunk: 
#             f.write(chunk) 
  
# print( "%s downloaded!\n"%file_name )

Sticks and Stone.mp4 downloaded!



## Define Download Functions

Import modules again to start from this cell:

In [6]:
import os
from pytube import YouTube

### **Video** *Download* and *Conversion* Functions

Download function:
* download first video stream with `pytube`

In [23]:
def download_video(youtube_url, song_title, file_path='.'):
  video_filename = song_title + ".mp4"
  yt_obj = YouTube(youtube_url)
  video = yt_obj.streams.first()
  video.download(output_path=file_path, filename=video_filename)

Conversion Function:
* run `ffmpeg` with `os.system()` module

In [25]:
def convert_to_audio(song_title, file_path='.', output_type="mp3"):
    vid = f'{song_title}.mp4'
    audio = f'{song_title}.{output_type}'

    # set variable with path to video and audio files
    vid_path = os.path.join(file_path, vid)
    audio_path = os.path.join(file_path, audio)
    
    # call `ffmpeg` to transform to output format
    os.system(f"ffmpeg -i '{vid_path}' '{audio_path}'")

/Users/colbyr/miniconda3/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### **Audio** Download Function
* Download audio using `.filter(only_audio)` option from `pytube`

In [26]:
def download_audio(youtube_url, song_title, file_path='.'):
  audio_filename = f"{song_title}.mp3"
  yt_obj = YouTube(youtube_url)
  audio = yt_obj.streams.filter(only_audio=True).first()
  audio.download(output_path=file_path, filename=audio_filename)

Test download function

In [37]:
youtube_video_url = "https://www.youtube.com/watch?v=YWhSQpUNGgY"

fname = "01 Sticks and Stones"

output_path = "./downloads"

download_audio(youtube_video_url, fname, output_path)

/Users/colbyr/miniconda3/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Edit ID3 Tags
Use either `mutagen` or `eyeD3`
* [mutagen](https://mutagen.readthedocs.io/en/latest/): `pip install mutagen`
* [eyeD3](https://eyed3.readthedocs.io/en/latest/): `pip install eyed3` 

### Check current metadata 
Use `ffprobe`, included in `ffmpeg` binary, to return the metadata currently attributed to the `.mp3` file

In [61]:
import subprocess
!ffprobe "./downloads/01 Sticks and Stones.mp3"

ffprobe version 4.3.1 Copyright (c) 2007-2020 the FFmpeg developers
  built with clang version 11.0.0
  configuration: --prefix=/Users/colbyr/miniconda3 --cc=x86_64-apple-darwin13.4.0-clang --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/Users/runner/miniforge3/conda-bld/ffmpeg_1609681034781/_build_env/bin/pkg-config
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from './downloads/01 Sticks an

/Users/colbyr/miniconda3/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Install and Test `eyeD3`

In [ ]:
!pip install eyed3

In [87]:
import eyed3

/Users/colbyr/miniconda3/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [97]:
path = "./downloads/01 Sticks and Stones.mp3"

audio_file = eyed3.mp3.Mp3AudioFile(path)
audio_file.initTag()
audio_file.tag.artist = "Kings Kaleidoscope"
audio_file.tag.album = "Beauty Between"
audio_file.tag.genre = "Alternative"
audio_file.tag.track_num = 1

audio_file.tag.save()

/Users/colbyr/miniconda3/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [98]:
!ffprobe './downloads/01 Sticks and Stones.mp3'

ffprobe version 4.3.1 Copyright (c) 2007-2020 the FFmpeg developers
  built with clang version 11.0.0
  configuration: --prefix=/Users/colbyr/miniconda3 --cc=x86_64-apple-darwin13.4.0-clang --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/Users/runner/miniforge3/conda-bld/ffmpeg_1609681034781/_build_env/bin/pkg-config
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x7f90f700ca00] Discarding ID3 tags be

In [120]:
af = eyed3.load(path)
af.tag.album = "Beauty Between"
af.tag.artist = "Kings Kaleidoscope" 

af.tag.save()

/Users/colbyr/miniconda3/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [121]:
print(af.tag.title)

None


In [108]:
!ffprobe "./downloads/01 Sticks and Stones.mp3"

ffprobe version 4.3.1 Copyright (c) 2007-2020 the FFmpeg developers
  built with clang version 11.0.0
  configuration: --prefix=/Users/colbyr/miniconda3 --cc=x86_64-apple-darwin13.4.0-clang --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/Users/runner/miniforge3/conda-bld/ffmpeg_1609681034781/_build_env/bin/pkg-config
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x7f9b2400be00] Discarding ID3 tags be

/Users/colbyr/miniconda3/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Using `Mutagen`
* `eyed3` seems to be having problems and documentation is inconsistent with available version(s) 
* also seems like iTunes mainly uses `.m4a` or `.m4p` file type, which `mutagen` explicitly supports

$\rightarrow$ switching to trying out `mutagen`

In [123]:
!pip install mutagen

In [125]:
import mutagen